<a href="https://colab.research.google.com/github/lancal/Tarea02Spark/blob/master/Tarea02Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [5]:
!ls

sample_data  spark-2.3.3-bin-hadoop2.7	spark-2.3.3-bin-hadoop2.7.tgz


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [7]:
!java --version


openjdk 11.0.3 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *
sqlSession = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [9]:
nums = sc.parallelize([1,2,3,4])
squared = nums.map(lambda x: x * x).collect()
for num in squared:
  print (num)

1
4
9
16


In [23]:
from google.colab import files

#drive.mount('drive');

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving master.csv to master.csv
User uploaded file "master.csv" with length 2228051 bytes


In [0]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


import math
import pandas as pd
import matplotlib.pylab as plt

Se crea RDD  y se almacena en cache, posteriormente se imprimen las 5 primeras lineas sin prepocesamiento ni limpieza.

In [24]:
suicidesRDD = sc.textFile("master.csv")
suicidesRDD.cache()
#suicidesRDD = suicidesRDD.flatMap(lambda t:t.split(","))
suicidesRDD.take(5)
#print (top5)

['country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_per_capita ($),generation',
 'Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,,796,Generation X',
 'Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,,796,Silent',
 'Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,,796,Generation X',
 'Albania,1987,male,75+ years,1,21800,4.59,Albania1987,,796,G.I. Generation']

Se Elimina el Encabezado:

In [25]:
header = suicidesRDD.first()
datalines = suicidesRDD.filter(lambda x : x != header)
#suicidesRDD.drop(' gdp_for_year ($) ').collect()
#print(suicidesRDD.collect())
#datalines = suicidesRDD.drop('gdp_for_year ($)').collect()
#datalines[9].dataType = IntegerType()
#suicidesRDD['gdp_for_year ($)'].dataType = IntegerType()


datalines.count()
datalines.take(5)

['Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,,796,Generation X',
 'Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,,796,Silent',
 'Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,,796,Generation X',
 'Albania,1987,male,75+ years,1,21800,4.59,Albania1987,,796,G.I. Generation',
 'Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,,796,Boomers']

Se define funcion para limpieza de cada linea del RDD

In [30]:
def cleanseRDD(inputStr) :
    if isinstance(inputStr, int) :
        return inputStr
      
    #Eliminar las comillas
    #repStr = inputStr.replace('\"','-')
      
    attList=inputStr.split(",")
    #
    if attList[2] == "male" :
         attList[2]="1"
    elif attList[2] == "female" :
         attList[2]="0"
        
    if attList[3] == "5-14 years":
         attList[3] = "0"
    elif attList[3] == "15-24 years" :
         attList[3]="1"
    elif attList[3] == "25-34 years" :
         attList[3]="2"
    elif attList[3] == "35-54 years" :
         attList[3]="3"
    elif attList[3] == "55-74 years" :
         attList[3]="4"
    elif attList[3] == "75+ years" :
         attList[3]="5"
        
    if attList[8] == "":
         attList[8] = "0"
        
    if attList[10] == "Boomers":
         attList[10] = "0"
    elif attList[10] == "G.I. Generation" :
         attList[10]="1"
    elif attList[10] == "Generation X" :
         attList[10]="2"
    elif attList[10] == "Generation Z" :
         attList[10]="3"
    elif attList[10] == "Millenials" :
         attList[10]="4"
    elif attList[10] == "Silent" :
         attList[10]="5"
        
        
    #attList[9].dataType = IntegerType()
  
    #Convert Drive to uppercase
    #attList[5] = attList[5].upper()
    return ",".join(attList)
  
cleanedData = datalines.map(cleanseRDD)
for line in cleanedData.take(500):
  print(line)

Albania,1987,1,1,21,312900,6.71,Albania1987,0,796,2
Albania,1987,1,3,16,308000,5.19,Albania1987,0,796,5
Albania,1987,0,1,14,289700,4.83,Albania1987,0,796,2
Albania,1987,1,5,1,21800,4.59,Albania1987,0,796,1
Albania,1987,1,2,9,274300,3.28,Albania1987,0,796,0
Albania,1987,0,5,1,35600,2.81,Albania1987,0,796,1
Albania,1987,0,3,6,278800,2.15,Albania1987,0,796,5
Albania,1987,0,2,4,257200,1.56,Albania1987,0,796,0
Albania,1987,1,4,1,137500,0.73,Albania1987,0,796,1
Albania,1987,0,0,0,311000,0,Albania1987,0,796,2
Albania,1987,0,4,0,144600,0,Albania1987,0,796,1
Albania,1987,1,0,0,338200,0,Albania1987,0,796,2
Albania,1988,0,5,2,36400,5.49,Albania1988,0,769,1
Albania,1988,1,1,17,319200,5.33,Albania1988,0,769,2
Albania,1988,1,5,1,22300,4.48,Albania1988,0,769,1
Albania,1988,1,3,14,314100,4.46,Albania1988,0,769,5
Albania,1988,1,4,4,140200,2.85,Albania1988,0,769,1
Albania,1988,0,1,8,295600,2.71,Albania1988,0,769,2
Albania,1988,0,4,3,147500,2.03,Albania1988,0,769,1
Albania,1988,0,2,5,262400,1.91,Albania1